# Exercise 2 Image Classificaton  
Environment: Python 3.7 & Pytorch 1.5.0+cpu on Windows 10 

Keypoint:
1. Self-defined dataset
2. Funetune classic pre-trained models
3. Models include AlexNet / MobileNet / MnasNet / ResNet / SqueezeNet / ShuffleNet
4. Datasets include hymenoptera_data / MNIST / Fashion_MNIST

## Content 1: Handwritting number recognition 

1. Use the MNIST provided by torchvision
2. Build the network by oneself 

### 1.1 Preparation: Load essential packages



In [12]:
import torch 
import torchvision 
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim 
from torchvision import datasets, transforms 
import torch.utils.data as tud 
import numpy as np 
import time

### 1.2 Data: Load image dataset 
Load pre-defined dataset or your customized dataset 

In [13]:
transform_1 = transforms.Compose([
                    transforms.ToTensor()
])

# transform_2 = transforms.Compose([
#                                      transforms.RandomResizedCrop(input_size),
#                                      transforms.RandomHorizontalFlip(),
#                                      transforms.ToTensor(),
#                                      transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
#                                  ])

transform_3 = transforms.Compose([
                    transforms.RandomHorizontalFlip(),
                    transforms.RandomGrayscale(),
                    transforms.ToTensor(),


])

trainset = datasets.MNIST(root='./data',train=True,download=True,transform=transform_3)

train_dataloader = tud.DataLoader(trainset, batch_size=100,shuffle=True,num_workers=0)

testset = datasets.MNIST(root='./data',train=False,download=True,transform=transform_1)

test_dataloader = tud.DataLoader(testset,batch_size=100,shuffle=False,num_workers=0)

### 1.3 Network configuration: AlexNet
Define network / Loss function / optimization algorithm 

In [14]:
# Define network in class 
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet,self).__init__()

        # The pic size in MNIST is 28x28, the input pic size of AlexNet is 227x227. So network depth and parameters need to be modified. 
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1) #AlexCONV1(3,96, k=11,s=4,p=0)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)#AlexPool1(k=3, s=2)
        self.relu1 = nn.ReLU()

        # self.conv2 = nn.Conv2d(96, 256, kernel_size=5,stride=1,padding=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)#AlexCONV2(96, 256,k=5,s=1,p=2)
        self.pool2 = nn.MaxPool2d(kernel_size=2,stride=2)#AlexPool2(k=3,s=2)
        self.relu2 = nn.ReLU()

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)#AlexCONV3(256,384,k=3,s=1,p=1)
        # self.conv4 = nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)#AlexCONV4(384, 384, k=3,s=1,p=1)
        self.conv5 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)#AlexCONV5(384, 256, k=3, s=1,p=1)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)#AlexPool3(k=3,s=2)
        self.relu3 = nn.ReLU()

        self.fc6 = nn.Linear(256*3*3, 1024)  #AlexFC6(256*6*6, 4096)
        self.fc7 = nn.Linear(1024, 512) #AlexFC6(4096,4096)
        self.fc8 = nn.Linear(512, 10)  #AlexFC6(4096,1000)

    def forward(self,x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.relu2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.pool3(x)
        x = self.relu3(x)
        x = x.view(-1, 256 * 3 * 3)#Alex: x = x.view(-1, 256*6*6)
        x = self.fc6(x)
        x = F.relu(x)
        x = self.fc7(x)
        x = F.relu(x)
        x = self.fc8(x)
        return x
    
net = AlexNet()   
    
# Define loss function 
loss_fn = nn.CrossEntropyLoss()

# Define optimization function 
optimizer = optim.SGD(net.parameters(),lr=0.01,momentum=0.9)

### 1.4 Nework Training 


In [15]:
# Define network training function 
def train(model,train_dataloader,loss_fn,optimizer,epoch):
    model.train()
    for idx, (data,label) in enumerate(train_dataloader):
        output = model(data)
        loss = loss_fn(output,label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if idx % 100 == 0 :
            print ("Train Epoch: {}, iteration: {}, loss: {}".format(
            epoch,idx,loss.item()))

### 1.5 Network evaluation 

In [16]:
def test(model,test_dataloader,loss_fn):
    model.eval()
    total_loss = 0 
    correct = 0 
    with torch.no_grad():
        for idx,(data,label) in enumerate(test_dataloader):
            output = model(data)
            loss = loss_fn(output,label)
            pred = output.argmax(dim=1)
            total_loss += loss 
            correct += (pred==label).sum()
        total_loss /= len(test_dataloader.dataset)
        acc  = correct.item()/len(test_dataloader.dataset)
        print('Test Loss:{}, Accuracy:{}\n'.format(total_loss,acc))

# main function 
num_epochs = 3 
for epoch in range(num_epochs):
    train(net,train_dataloader,loss_fn,optimizer,epoch)
    test(net,test_dataloader,loss_fn)

Train Epoch: 0, iteration: 0, loss: 2.3051509857177734
Train Epoch: 0, iteration: 100, loss: 2.2957708835601807
Train Epoch: 0, iteration: 200, loss: 2.285892963409424
Train Epoch: 0, iteration: 300, loss: 1.3233134746551514
Train Epoch: 0, iteration: 400, loss: 0.5398569703102112
Train Epoch: 0, iteration: 500, loss: 0.3060709238052368
Test Loss:0.00242247455753386, Accuracy:0.9189

Train Epoch: 1, iteration: 0, loss: 0.19003471732139587
Train Epoch: 1, iteration: 100, loss: 0.18734055757522583
Train Epoch: 1, iteration: 200, loss: 0.19276869297027588
Train Epoch: 1, iteration: 300, loss: 0.18399423360824585
Train Epoch: 1, iteration: 400, loss: 0.15822146832942963
Train Epoch: 1, iteration: 500, loss: 0.15540261566638947
Test Loss:0.0014110058546066284, Accuracy:0.954

Train Epoch: 2, iteration: 0, loss: 0.1525941789150238
Train Epoch: 2, iteration: 100, loss: 0.10462642461061478
Train Epoch: 2, iteration: 200, loss: 0.15674740076065063
Train Epoch: 2, iteration: 300, loss: 0.0989114

### 1.6 Model storage 

In [ ]:
best_valid_acc = 0 
for epoch in range(num_epochs):
    train(net,train_dataloader,loss_fn,optimizer,epoch)
    acc = test(model,test_dataloader,loss_fn)
    if acc > best_valid_acc:
        best_valid_acc = acc
        now = time.strftime("%Y-%m-%d-%H_%M_%S",time.localtime(time.time())) 
        fname="./trained/"+now+r"Best_MNIST_AlexNet.pth"
        torch.save(net.state_dict(),fname)

## Content 1-2: Fashion MNIST with AlexNet


In [20]:
# Fashion_MNIST need to be downloaded in preparation

batch_size = 32 

train_dataloader = tud.DataLoader(
    datasets.FashionMNIST("./datasets/fashion_mnist_data",train=True,download=True,
                         transform=transforms.Compose([
                             transforms.ToTensor(),
                             transforms.Normalize(mean=(0.2860402,),std=(0.3530239,))
                         ])),batch_size = batch_size,shuffle = True)

test_dataloader = tud.DataLoader(
    datasets.FashionMNIST("./datasets/fashion_mnist_data",train=False,download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize(mean=(0.2860402,),std=(0.3530239,))
                   ])),batch_size=batch_size) 

lr = 0.01
momentum = 0.5
net = AlexNet()

optimizer = optim.SGD(net.parameters(),lr=lr,momentum=momentum)
num_epochs = 2
for epoch in range(num_epochs):
    train(net,train_dataloader,loss_fn,optimizer,epoch)
    test(net,test_dataloader,loss_fn)
    
torch.save(net.state_dict(),"./trained/fashion_mnist_cnn.pth")

Train Epoch: 0, iteration: 0, loss: 2.3075597286224365
Train Epoch: 0, iteration: 100, loss: 2.2923483848571777
Train Epoch: 0, iteration: 200, loss: 2.240149736404419
Train Epoch: 0, iteration: 300, loss: 1.28989839553833
Train Epoch: 0, iteration: 400, loss: 0.8979262709617615
Train Epoch: 0, iteration: 500, loss: 0.8208087086677551
Train Epoch: 0, iteration: 600, loss: 0.8645915985107422
Train Epoch: 0, iteration: 700, loss: 0.7522311210632324
Train Epoch: 0, iteration: 800, loss: 0.6402029395103455
Train Epoch: 0, iteration: 900, loss: 0.5410202741622925
Train Epoch: 0, iteration: 1000, loss: 0.8181690573692322
Train Epoch: 0, iteration: 1100, loss: 0.6852773427963257
Train Epoch: 0, iteration: 1200, loss: 0.6652778387069702
Train Epoch: 0, iteration: 1300, loss: 0.6709299683570862
Train Epoch: 0, iteration: 1400, loss: 0.5586152672767639
Train Epoch: 0, iteration: 1500, loss: 1.1550471782684326
Train Epoch: 0, iteration: 1600, loss: 0.5558242797851562
Train Epoch: 0, iteration: 17

NameError: name 'model' is not defined

## Content 2: Classification of ant and bee 
1. Customized dataset:hymenoptera_data
2. Finetune based on existing model(ResNet-18/MobileNet/ShuffleNet/...)

### 1. Package Perparation 

 

In [1]:
import os
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models, datasets, transforms
import torch.utils.data as tud
import numpy as np

### 2. Data perparation 

In [2]:
# The download link of hymenoptera_data: https://download.pytorch.org/tutorial/hymenoptera_data.zip

data_dir = './datasets/hymenoptera_data'

num_class =2 
input_size = 224 
batch_size = 32 

# all_imgs = datasets.ImageFolder(os.path.join(data_dir,"train"),
#                                transform = transforms.Compose([
#                                    transforms.RandomResizedCrop(input_size),
#                                    transforms.RandomHorizontalFlip(),
#                                    transforms.ToTensor()
#                                ]))
# loader = tud.DataLoader(all_imgs,batch_size=batch_size,shuffle=True)

train_imgs = datasets.ImageFolder(os.path.join(data_dir,"train"),
                                transform=transforms.Compose([
                                    transforms.RandomResizedCrop(input_size),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406],[0.229,0.224,0.225])
                                ]))
train_dataloader = tud.DataLoader(train_imgs,batch_size=batch_size,shuffle=True)

test_imgs = datasets.ImageFolder(os.path.join(data_dir,"val"),
                                transform=transforms.Compose([
                                    transforms.Resize(input_size),  
                                    transforms.CenterCrop(input_size),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406],[0.229,0.224,0.225])
                                ]))
test_dataloader = tud.DataLoader(test_imgs,batch_size=batch_size)

### 3. Network configuration 

In [10]:
# Define network 
def initialize_model (model_name,num_class,use_pretrained=True,feature_extract=True):
    if model_name == 'resnet18': 
        # Option1:Download the pth file 
        model_ft = models.resnet18(pretrained=use_pretrained)        
    `   # Option2: Use the pth file stored in local computer 
#         model_ft = models.resnet18(pretrained=False)
#         model_ft.load_state_dict(torch.load('./trained/resnet18-5c106cde.pth'))        
        if feature_extract:
            for param in model_ft.parameters():
                param.requires_grad = False 
        num_ftrs = model_ft.fc.in_features 
        model_ft.fc = nn.Linear(num_ftrs,num_class)
        
    elif model_name == 'squeezenet1_0': 
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)             
        if feature_extract:
            for param in model_ft.parameters():
                param.requires_grad = False 
        model_ft.classifier[1] = nn.Conv2d(512,num_class,kernel_size=(1,1),stride=(1,1))
        model_ft.num_classes=num_class   
        
    elif model_name == 'mnasnet0_5':
        model_ft = models.mnasnet0_5(pretrained=use_pretrained)
        if feature_extract:
            for param in model_ft.parameters():
                param.requires_grad = False         
        model_ft.classifier[1].out_features = num_class
        for param in model_ft.parameters():
            param.requires_grad = True
        
    elif model_name == 'mobilenet_v2': 
        model_ft = models.mobilenet_v2(pretrained=use_pretrained)
        if feature_extract:
            for param in model_ft.parameters():
                param.requires_grad = False 
        model_ft.classifier[1].out_features = num_class
        for param in model_ft.parameters():
            param.requires_grad = True
     
    elif model_name == 'shufflenet_v2_x0_5': 
        model_ft = models.shufflenet_v2_x0_5(pretrained=use_pretrained)
        if feature_extract:
            for param in model_ft.parameters():
                param.requires_grad = False 
        num_ftrs = model_ft.fc.in_features 
        model_ft.fc = nn.Linear(num_ftrs,num_class)  
        
    else:
        print('Model not implemented.')
        return None 
    return model_ft

# To use different pretrained model, change the 'resnet18' to different model_name in initialize_model
model_ft = initialize_model("resnet18",2,use_pretrained=True,feature_extract=True)

# More pretrained models, check:https://pytorch.org/docs/master/torchvision/models.html?highlight=torchvision%20models
# Selected Pre-trained models(resnet18/squeezenet1_0/mnasnet0_5/mobilenet_v2/shufflenet_v2_x0_5) in this code whose .pth file to easy to download 

# Define Loss
loss_fn = nn.CrossEntropyLoss()

# Define optimization function, select optimal parameter for different model checked

# Parameters for resnet18 / squeezenet1_0 / mobilenet_v2
lr = 0.01
momentum = 0.5

# Parameters for shufflenet_v2_x0_5
# lr = 0.05  
# momentum = 0.5

# Parameters for mnasnet
# lr = 0.1 
# momentum = 0.9

optimizer = optim.SGD(model_ft.parameters(),lr=lr,momentum=momentum)

Sequential(
  (0): Dropout(p=0.2, inplace=True)
  (1): Linear(in_features=1280, out_features=2, bias=True)
)


### 4. Network training 

In [7]:
def train_model(model,train_dataloader,loss_fn,optimizer,epoch):
    model.train()
    total_loss = 0.
    total_corrects = 0.
    for idx, (inputs, labels) in enumerate(train_dataloader):
        outputs = model(inputs)
        loss = loss_fn(outputs,labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        preds = outputs.argmax(dim=1)
        total_loss += loss.item() * inputs.size(0)
        total_corrects += torch.sum(preds.eq(labels))
    epoch_loss = total_loss / len(train_dataloader.dataset)
    epoch_accuracy = total_corrects / len(train_dataloader.dataset)
    print("Epoch:{}, Training Loss:{:.4f}, Traning Acc:{:.4f}".format(epoch,epoch_loss,epoch_accuracy))  
      
            

### 5. Model evaluation  

In [8]:
def test_model(model,test_dataloader,loss_fn):
    model.eval()
    total_loss = 0.
    total_corrects = 0.
    with torch.no_grad():
        for idx, (inputs, labels) in enumerate(test_dataloader):
            outputs = model(inputs)
            loss = loss_fn(outputs,labels)
            preds = outputs.argmax(dim=1)
            total_loss += loss.item() * inputs.size(0)
            total_corrects += torch.sum(preds.eq(labels))
    epoch_loss = total_loss / len(test_dataloader.dataset)
    epoch_accuracy = total_corrects / len(test_dataloader.dataset)
    print("Test Loss:{:.4f}, Test Acc:{:.4f}".format(epoch_loss,epoch_accuracy))  
    return epoch_accuracy 

In [11]:
num_epochs = 5
for epoch in range(num_epochs):
   train_model(model_ft,train_dataloader,loss_fn,optimizer,epoch)
   test_model(model_ft,test_dataloader,loss_fn)

Epoch:0, Training Loss:6.0485, Traning Acc:0.1025
Test Loss:2.8012, Test Acc:0.1765
Epoch:1, Training Loss:0.7612, Traning Acc:0.7459
Test Loss:0.5049, Test Acc:0.7843
Epoch:2, Training Loss:0.3640, Traning Acc:0.8361
Test Loss:0.4426, Test Acc:0.8301
Epoch:3, Training Loss:0.3488, Traning Acc:0.8566
Test Loss:0.3897, Test Acc:0.8366
Epoch:4, Training Loss:0.2931, Traning Acc:0.8852
Test Loss:0.3080, Test Acc:0.8954
